In [1]:
import numpy as np
from random import randrange, random


class State:

    def __init__(self, positions=None, board_size=8):
        '''
        Initialise a new state with random placement of queens if no positions are given,
        or generate a board with a set of 2d vectors for the location of each queen
        '''
        if positions is None:
            self.positions = []
            self.board = np.zeros((board_size, board_size))
            
            for queen in range(board_size):
                row = randrange(1, board_size)
                col = randrange(1, board_size)

                while(self.board[row, col] == 1):
                    row = randrange(1, board_size)
                    col = randrange(1, board_size)

                self.board[row, col] = 1
                self.positions.append((row, col))
                
        else:
            self.positions = positions
            self.board = np.zeros((len(positions), len(positions)))
            
            for pos in positions:
                r, c = pos
                self.board[r, c] = 1
    
    def check_rows(self) -> int:
        sigma = 0
        for row in self.board:
            if np.sum(row) > 1:
                sigma += np.sum(row)
        return int(sigma)
            
    def check_cols(self) -> int:
        sigma = 0
        for col in self.board.T:
            if np.sum(col) > 1:
                sigma += np.sum(col)
        return int(sigma)
    
    def check_diagonals(self) -> bool:
        M = self.board
        M_f = np.fliplr(M)
        n = len(M)
        
        sigma = 0
        for i in range(-n, n, 1):
            if np.sum(M.diagonal(i)) > 1:
                sigma += np.sum(M.diagonal(i))
            if np.sum(M_f.diagonal(i)) > 1:
                sigma += np.sum(M_f.diagonal(i))
                
        return int(sigma)
    
        
    def copy(self):
        '''
        Creates a deep copy of the current state
        '''
        return State(positions=self.positions.copy())

    def __str__(self) -> None:
        return str(f"{self.board}\n{self.positions}\n")



In [2]:
def evaluate(state: State) -> int:
    '''
    Evaluates the state of the board and returns an integer value for the number of
    faults that the board has.  A fault defined as two or more quees on the same horizontal,
    vertical or diagonal.
    '''
    return state.check_rows() + state.check_cols() + state.check_diagonals()


In [3]:
def breed(parents: list) -> list:
    '''
    Take a population and breed every combination of parents to create children
    The output will be a list of n^2 children, where n is the original number of parents
    '''
    children = []
    
    for i in range(len(parents)):
        
        for j in range(len(parents)):
            
            if i == j:
                children.append(parents[i])
            
            else:
                children.append(mutate(crossover(parents[i], parents[j])))
                
    return children

In [4]:
def crossover(parent_a: State, parent_b: State) -> State:
    '''
    Create a new state that inherits about 50% of it's atributes from each parent
    '''
    child = []
    for i in range(len(parent_a.positions)):
        
        if random() > 0.5:
            child.append(parent_a.positions[i])
        else:
            child.append(parent_b.positions[i])
    
    return State(child)
        

In [5]:
alpha = 0.1

def mutate(state: State) -> State:
    '''
    Return a new state that is a mutated version of the state passed in, where mutation is defined 
    as a random change in any integer in the positions (or "DNA") of the state 
    '''
    n = len(state.board)
    positions = state.positions.copy() 
    
    for i in range(len(positions)):
        
        if random() < alpha:
            
            r = randrange(1, n)
            c = randrange(1, n)
            
            while(state.board[r, c] == 1):  # Ensure we aren't overwriting another queen
                r = randrange(1, n)
                c = randrange(1, n)
                
            positions[i] = (r, c)
    
    return State(positions=positions)
    

In [6]:
def select(gen: list) -> list:
    '''
    Select the top 10% of the population, with the lowest evaluation function being the "fittest"
    '''
    gen.sort(key=evaluate)
    n = int(len(gen) / 10)
    return gen[:n]
    

In [10]:
from time import time

gen = [State() for i in range(100)]
best = 1000
count = 1

while best != 0:
    start = time()
    gen = select(gen)
    best = evaluate(gen[0])
    gen = breed(gen)
    
    print(f"Gen {count}: {best}  {time() - start}")
    count += 1

Gen 1: 12  0.017592430114746094
Gen 2: 8  0.01587700843811035
Gen 3: 6  0.016196012496948242
Gen 4: 4  0.015541315078735352
Gen 5: 2  0.016091346740722656
Gen 6: 2  0.015549898147583008
Gen 7: 0  0.015796184539794922
